In [1]:
# Aim: Scrape data from Wikipedia to create a text corpus
# Process:
# Result:

In [2]:
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
import os
import json

In [3]:
class WikiCategoryDataScrapper:
    def __init__(self, url, articlePrefix, dataPath=None):
        self.categoryURL = url
        self.articlePrefix = prefix
        self.categoryPageSoup = self.soup(self.request(self.categoryURL))
        self.meta = {}
        
        if dataPath is None:
            os.mkdir('Data')
            dataPath  = 'Data'
            
        self.index = self.letterIndices()
        urls = {}
        print('Identifying articles to scrape.')
        for letter in tqdm(self.index.keys()):
            urls[letter] = self.letterLinks(letter)
        
        for letter in urls.keys():
            print(f'Letter {letter} progress...')
            os.mkdir(f'{dataPath}\{letter}')
            for articleTitle in tqdm(urls[letter].keys()):
                articleURL = urls[letter][articleTitle]
                article = self.scrapeArticle(articleURL)
                if '\\' in articleTitle:
                    articlePath = f'{dataPath}/{letter}/{articleTitle.pop}.txt'    
                articlePath = f'{dataPath}/{letter}/{articleTitle}.txt'
                with open(articlePath,'w',encoding="utf-8") as articleFile:
                    articleFile.write(article)
                self.meta[articleTitle] = {
                    'url':urls[letter][articleTitle],
                    'path':articlePath
                }
        
        print(f'Scraped {len(self.meta.keys())} articles.')
        metaData = json.dump(self.meta,indent=4)
        with opern(f'{dataPath}\info.json',w) as metaFile:
            metaFile.write(metaData)
        print(f'Meta data present in file {dataPath}\info.json')
    
    def request(self,url):
        response = requests.get(url)
        return response

    def soup(self, response):
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    
    def letterIndices(self):
        index = {}
        soups = self.categoryPageSoup.body.find(id='mw-pages').find(class_='CategoryIndex').find_all('li')[2:]
        for soup in soups:
            index[soup.text] = soup.a['href']
        return index
    
    def letterLinks(self,letter):
        letterResponse = self.request(self.index[letter])
        letterSoup = self.soup(letterResponse).body.find(class_='mw-category-group')
        articles =  {}
        articleSoups = letterSoup.find_all('li')
        for articleSoup in articleSoups:
            articles[articleSoup.text] = self.articlePrefix + articleSoup.a['href']
        return articles

    def scrapeArticle(self,articleURL):
        articleResponse = self.request(articleURL)
        articleSoup = self.soup(articleResponse)
        contentSoup = articleSoup.find(class_='mw-parser-output')
        content = ''.join([line.text for line in articleSoup.find(class_='mw-parser-output').find_all(['p','ul','ol'])])
        return content

In [4]:
url = 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&pageuntil=Essaim+de+drones#mw-pages'
prefix = 'https://fr.wikipedia.org'
scrapper = WikiCategoryDataScrapper(url,prefix)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'Data'